In [1]:
from music.models import Album, Song

In [7]:
Album.objects.bulk_create([
    Album(album_title="Humbug", artist="Arctic Monkeys"),
    Album(album_title="Is This It", artist="The Strokes"),
    Album(album_title="Happier Than Ever", artist="Billie Eilish"),
    Album(album_title="Dreamland", artist="Glass Animals"),
])

[<Album: Humbug by Arctic Monkeys>,
 <Album: Is This It by The Strokes>,
 <Album: Happier Than Ever by Billie Eilish>,
 <Album: Dreamland by Glass Animals>]

In [8]:
humbug = Album.objects.get(album_title="Humbug", artist="Arctic Monkeys")
is_this_it = Album.objects.get(album_title="Is This It", artist="The Strokes")
hte = Album.objects.get(album_title="Happier Than Ever", artist="Billie Eilish")
dreamland = Album.objects.get(album_title="Dreamland", artist="Glass Animals")

In [4]:
Song.objects.create(song_title="Drag Queen")
Song.objects.create(song_title="Everything I Wanted")
Song.objects.create(song_title="I Dont Wanna Talk (I Just Wanna Dance)")

<Song: I Dont Wanna Talk (I Just Wanna Dance)>

In [9]:
is_this_it.songs.create(song_title="Last Nite")

<Song: Last Nite>

In [10]:
hte.songs.create(song_title="NDA")

<Song: NDA>

In [11]:
dreamland.songs.create(song_title="Tangerine")

<Song: Tangerine>

In [15]:
songs = Song.objects.all()
for song in songs:
    print(song.album.album_title)

Is This It
Happier Than Ever
Dreamland
